In [1]:
!pip install tensorflow==2.10.0
!pip install git+https://github.com/keras-team/keras-nlp.git --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.0 MB 19 kB/s 
     |████████████████████████████████| 1.7 MB 53.8 MB/s 
     |████████████████████████████████| 5.9 MB 56.0 MB/s 
     |████████████████████████████████| 438 kB 64.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully u

In [3]:
import os
import keras_nlp
import tensorflow as tf
from tensorflow import keras

In [5]:
print(tf.__version__)
print(keras_nlp.__version__)

2.10.0
0.3.0


In [6]:
# Data
BATCH_SIZE = 64
SEQ_LEN = 128
MIN_TRAINING_SEQ_LEN = 450

# Model
EMBED_DIM = 256
FEED_FORWARD_DIM = 256
NUM_HEADS = 3
NUM_LAYERS = 2
VOCAB_SIZE = 5000  # Limits parameters in model.

# Training
EPOCHS = 6

# Inference
NUM_TOKENS_TO_GENERATE = 10

In [4]:
keras.utils.get_file(
    origin="https://dldata-public.s3.us-east-2.amazonaws.com/simplebooks.zip",
    extract=True,
)
dir = os.path.expanduser("~/.keras/datasets/simplebooks/")

282386239/282386239 [==============================] - 24s 0us/step


In [7]:
!head -n20 ~/.keras/datasets/simplebooks/simplebooks-2-raw/train.txt

More Jataka Tales

By

Ellen C. Babbitt



I

The Girl Monkey And The String Of Pearls


One day the king went for a long walk in the woods. When he came back to his own garden, he sent for his family to come down to the lake for a swim.

When they were all ready to go into the water, the queen and her ladies left their jewels in charge of the servants, and then went down into the lake.

As the queen put her string of pearls away in a box, she was watched by a Girl Monkey who sat in the branches of a tree near-by. This Girl Monkey wanted to get the queen's string of pearls, so she sat still and watched, hoping that the servant in charge of the pearls would go to sleep.

At first the servant kept her eyes on the jewel-box. But by and by she began to nod, and then she fell fast asleep.


In [8]:
NUM_ITERS = 10

dataset = tf.data.TextLineDataset(dir + "simplebooks-92-raw/train.txt")
print(dataset)

<TextLineDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [9]:
for i, element in enumerate(dataset):
    print(element)

    if i > NUM_ITERS:
        break

tf.Tensor(b"Dave Darrin's Second Year At Annapolis", shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'Or', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'Two Midshipmen As Naval Academy "Youngsters"', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'By', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'H. Irving Hancock', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)


In [10]:
for i, element in enumerate(dataset.as_numpy_iterator()):
    print(element)
    if i > NUM_ITERS:
        break

b"Dave Darrin's Second Year At Annapolis"
b''
b'Or'
b''
b'Two Midshipmen As Naval Academy "Youngsters"'
b''
b'By'
b''
b'H. Irving Hancock'
b''
b''
b''


In [11]:
filtered_dataset = dataset.filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)

for i, element in enumerate(filtered_dataset.as_numpy_iterator()):
    print(element)

    if i > NUM_ITERS:
        break

b"Readers of that preceding volume will recall how Dave Darrin and Dan Dalzell entered the United States Naval Academy, one appointed by a Congressman and the other by a United States Senator. Such readers will remember the difficult time that Dave and Dan had in getting through the work of the first hard, grinding year. They will also recall how Dave Darrin, when accused of treachery to his classmates, patiently bided his time until he, with the aid of some close friends, was able to demonstrate his innocence. Our readers will also remember how two evil-minded members of the then fourth class plotted to increase Damn's disgrace and to drive him out of the brigade; also how these two plotters, Midshipmen Henkel and Brimmer, were caught in their plotting and were themselves forced out of the brigade. Our readers know that before the end of the first year at the Naval Academy, Dave had fully reinstated himself in the esteem of his manly classmates, and how he quickly became the most popu

In [12]:
batched_dataset = dataset.filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN).batch(2)

for i, element in enumerate(batched_dataset.as_numpy_iterator()):
    print(element)
    
    if i > NUM_ITERS:
        break

[b"Readers of that preceding volume will recall how Dave Darrin and Dan Dalzell entered the United States Naval Academy, one appointed by a Congressman and the other by a United States Senator. Such readers will remember the difficult time that Dave and Dan had in getting through the work of the first hard, grinding year. They will also recall how Dave Darrin, when accused of treachery to his classmates, patiently bided his time until he, with the aid of some close friends, was able to demonstrate his innocence. Our readers will also remember how two evil-minded members of the then fourth class plotted to increase Damn's disgrace and to drive him out of the brigade; also how these two plotters, Midshipmen Henkel and Brimmer, were caught in their plotting and were themselves forced out of the brigade. Our readers know that before the end of the first year at the Naval Academy, Dave had fully reinstated himself in the esteem of his manly classmates, and how he quickly became the most pop

In [13]:
# 학습 데이터셋 파이프라인
raw_train_ds = (
    tf.data.TextLineDataset(dir + "simplebooks-92-raw/train.txt")
    .filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)
    .batch(BATCH_SIZE)
    .shuffle(buffer_size=256)
)

# 평가 데이터셋 파이프라인
raw_val_ds = (
    tf.data.TextLineDataset(dir + "simplebooks-92-raw/valid.txt")
    .filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)
    .batch(BATCH_SIZE)
)

In [14]:
# 토크나이저 사전을 학습합니다.
vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    raw_train_ds,
    vocabulary_size=VOCAB_SIZE,
    lowercase=True,
    reserved_tokens=["[PAD]", "[UNK]", "[BOS]"],
)

In [15]:
# 이게 아마 클래스를 선언하는 것인듯(객체 생성)

tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    sequence_length=SEQ_LEN,
    lowercase=True
)

In [16]:
print([tokenizer.id_to_token(t) for t in tokenizer.tokenize('hello world!').numpy().tolist()])

['he', '##ll', '##o', 'world', '!', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [17]:
start_packer = keras_nlp.layers.StartEndPacker(
    sequence_length=SEQ_LEN,
    start_value=tokenizer.token_to_id("[BOS]"),
)

In [18]:
tokens = tokenizer('hello world')
start_packer(tokens)

<tf.Tensor: shape=(128,), dtype=int32, numpy=
array([   2,  103, 1520,  291,  394,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0], dtype=int32)>

In [19]:
def preprocess(inputs):
    outputs = tokenizer(inputs)
    features = start_packer(outputs)
    labels = outputs
    return features, labels

In [20]:
preprocess("This is test sentence.")

(<tf.Tensor: shape=(128,), dtype=int32, numpy=
 array([   2,  139,  124,   56,  410,  375, 1121,   15,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0], dtype=int32)>,
 <tf.Tensor: shape=(128,), dtype=int32, numpy=
 array([ 139,  124,   56, 

In [21]:
train_ds = raw_train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)
val_ds = raw_val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)

In [22]:
inputs = keras.layers.Input(shape=(None,), dtype=tf.int32)

In [23]:
# Embedding.
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=SEQ_LEN,
    embedding_dim=EMBED_DIM,
    mask_zero=True, 
)
x = embedding_layer(inputs)

In [24]:
for _ in range(NUM_LAYERS):
    decoder_layer = keras_nlp.layers.TransformerDecoder(
        num_heads=NUM_HEADS,
        intermediate_dim=FEED_FORWARD_DIM,
    )
    x = decoder_layer(x) 

In [25]:
outputs = keras.layers.Dense(VOCAB_SIZE)(x)

In [26]:
model = keras.Model(inputs=inputs, outputs=outputs) # 모델 설정은 이렇게 간단하다.

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# 메트릭 함수. Negative log likelihood or crossentorpy 사용

perplexity = keras_nlp.metrics.Perplexity(from_logits=True, mask_token_id=0)
# 대표적인 스코어로는 perplexity, BLEU, ROUGE를 사용.

model.compile(optimizer="adam", loss=loss_fn, metrics=[perplexity])

In [27]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 256)        1312768   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, None, 256)        394749    
 ormerDecoder)                                                   
                                                                 
 transformer_decoder_1 (Tran  (None, None, 256)        394749    
 sformerDecoder)                                                 
                                                                 
 dense (Dense)               (None, None, 5000)        128500

In [39]:
model.fit(train_ds, validation_data=val_ds, verbose=2, epochs=EPOCHS)
# 1400초 이상 예상, 실제로는 1471초 소요

Epoch 1/6
3169/3169 - 232s - loss: 4.5421 - perplexity: 99.5348 - val_loss: 4.0073 - val_perplexity: 62.6560 - 232s/epoch - 73ms/step
Epoch 2/6
3169/3169 - 227s - loss: 4.0069 - perplexity: 57.9084 - val_loss: 3.8749 - val_perplexity: 54.5767 - 227s/epoch - 72ms/step
Epoch 3/6
3169/3169 - 227s - loss: 3.8897 - perplexity: 51.4289 - val_loss: 3.8042 - val_perplexity: 50.9426 - 227s/epoch - 72ms/step
Epoch 4/6
3169/3169 - 227s - loss: 3.8261 - perplexity: 48.2199 - val_loss: 3.7550 - val_perplexity: 48.3532 - 227s/epoch - 72ms/step
Epoch 5/6
3169/3169 - 228s - loss: 3.7842 - perplexity: 46.2218 - val_loss: 3.7305 - val_perplexity: 47.2047 - 228s/epoch - 72ms/step
Epoch 6/6
3169/3169 - 227s - loss: 3.7526 - perplexity: 44.7616 - val_loss: 3.7350 - val_perplexity: 47.4588 - 227s/epoch - 72ms/step


In [50]:
model.save('/content/drive/MyDrive/keras_nlp_gpt_example_ep6.h5')

In [52]:
# 추가 학습 진행 ...

model_loaded = tf.keras.models.load_model('/content/drive/MyDrive/keras_nlp_gpt_example_ep6.h5')

model_loaded.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 256)        1312768   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, None, 256)        394749    
 ormerDecoder)                                                   
                                                                 
 transformer_decoder_1 (Tran  (None, None, 256)        394749    
 sformerDecoder)                                                 
                                                                 
 dense (Dense)               (None, None, 5000)        128500

In [53]:
model_loaded.fit(train_ds, validation_data=val_ds, verbose=2, epochs=9)
model_loaded.save('/content/drive/MyDrive/keras_nlp_gpt_example_ep15.h5')
# 37분, 2223초

Epoch 1/9
3169/3169 - 230s - loss: 3.7301 - perplexity: 43.7561 - val_loss: 3.7053 - val_perplexity: 45.8648 - 230s/epoch - 73ms/step
Epoch 2/9
3169/3169 - 227s - loss: 3.7096 - perplexity: 42.8543 - val_loss: 3.6889 - val_perplexity: 45.2284 - 227s/epoch - 72ms/step
Epoch 3/9
3169/3169 - 227s - loss: 3.6919 - perplexity: 42.0932 - val_loss: 3.6798 - val_perplexity: 44.7842 - 227s/epoch - 72ms/step
Epoch 4/9
3169/3169 - 228s - loss: 3.6778 - perplexity: 41.4983 - val_loss: 3.6505 - val_perplexity: 43.4797 - 228s/epoch - 72ms/step
Epoch 5/9
3169/3169 - 227s - loss: 3.6657 - perplexity: 40.9912 - val_loss: 3.6508 - val_perplexity: 43.5056 - 227s/epoch - 72ms/step
Epoch 6/9
3169/3169 - 228s - loss: 3.6548 - perplexity: 40.5422 - val_loss: 3.6415 - val_perplexity: 43.0889 - 228s/epoch - 72ms/step
Epoch 7/9
3169/3169 - 227s - loss: 3.6457 - perplexity: 40.1706 - val_loss: 3.6221 - val_perplexity: 42.1839 - 227s/epoch - 72ms/step
Epoch 8/9
3169/3169 - 227s - loss: 3.6368 - perplexity: 39.809

# 텍스트 생성

In [41]:
prompt_tokens = tf.convert_to_tensor([tokenizer.token_to_id("[BOS]")])

In [42]:
def token_logits_fn(inputs):
    cur_len = inputs.shape[1]
    output = model(inputs)
    return output[:, cur_len - 1, :]  # return next token logits


In [30]:
# before model. fit
output_tokens = keras_nlp.utils.greedy_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
)

txt = tokenizer.detokenize(output_tokens)
print(f"Before model fitting, Greedy search generated text: \n{txt}\n")

Greedy search generated text: 
b'[BOS]rf feel beasts shouldnoriesaming special stand huge'



In [43]:
# After model fit
output_tokens = keras_nlp.utils.greedy_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
)

txt = tokenizer.detokenize(output_tokens)
print(f"Greedy search generated text: \n{txt}\n")

Greedy search generated text: 
b'[BOS] " i am not going to be a very'



In [33]:
# Before model fit

output_tokens = keras_nlp.utils.beam_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    num_beams=20,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Beam search generated text: \n{txt}\n")

Beam search generated text: 
b'[BOS]rf feel beasts shouldnoriesaming specialzyless'



In [44]:
# After model fit

output_tokens = keras_nlp.utils.beam_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    num_beams=20,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Beam search generated text: \n{txt}\n")


Beam search generated text: 
b'[BOS] " i don \' t know , " said'



In [34]:
# before model fit

output_tokens = keras_nlp.utils.random_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Random search generated text: \n{txt}\n")

Random search generated text: 
b'[BOS]mons give sunlight horsesch holidaysirt bare rescue'



In [45]:
# after model fit
output_tokens = keras_nlp.utils.random_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Random search generated text: \n{txt}\n")

Random search generated text: 
b'[BOS] although theirs were yet peaceful , the scene'



In [35]:
# before model fit

output_tokens = keras_nlp.utils.top_k_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    k=10,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-K search generated text: \n{txt}\n")

Top-K search generated text: 
b'[BOS] cup hurt discipline parts risingant fair originalife'



In [46]:
# after model fit

output_tokens = keras_nlp.utils.top_k_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    k=10,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-K search generated text: \n{txt}\n")

Top-K search generated text: 
b'[BOS] " there are many a little people who have'



In [37]:
# before model fit
output_tokens = keras_nlp.utils.top_p_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    p=0.1,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-P search generated text: \n{txt}\n")

Top-P search generated text: 
b"[BOS]shipcle' camp hour clockessallwin"



In [47]:
# after model fit

output_tokens = keras_nlp.utils.top_p_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    p=0.1,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-P search generated text: \n{txt}\n")

Top-P search generated text: 
b'[BOS] " i am not sure that i am not'



In [38]:
# before model fit

output_tokens = keras_nlp.utils.top_p_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    p=0.6,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-P search generated text: \n{txt}\n")

Top-P search generated text: 
b'[BOS] many herdius\xc3\xb9 beardp unlike laughing trail'



In [48]:
# after model fit

output_tokens = keras_nlp.utils.top_p_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    p=0.6,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-P search generated text: \n{txt}\n")

Top-P search generated text: 
b"[BOS] it was but eight o ' clock , as"

